# Data Processing

In Chapter 2 we have seen several data structures that represent sequential data implicitly (calculation of the dataset on demand) such as iterators, iterables, generators and streams.


## Declarative Programming & SQL

In addition to streams, data values are often stored in large repositories called databases. A **database** consists of a data store containing the data values along with an interface for retrieving and transforming those values. Each value stored in a database is called a **record**. Records with similar structure are grouped into **tables**. Records are retrieved and transformed using **queries**, which are statements in a **query language**. By far the most ubiquitous query language in use today is called Structured Query Language or SQL (pronounced "sequel").

SQL is an example of a declarative programming language. Statements do not describe computations directly, but instead describe the desired result of some computation. It is the role of the **query interpreter** of the database system to design and perform a computational process to produce such a result.

The SQL language is standardized, but most database systems implement some custom variant of the language that is endowed with proprietary features. In this text, we will describe a small subset of SQL as it is implemented in Sqlite.

### Tables

A table, also called a **relation**, has a fixed number of named and typed columns. Each row of a table represents a data record and has one value for each column.

A table with a single row can be created in the SQL language using a `select` statement, in which the row values are separated by commas and the column names follow the keyword `as`. All SQL statements end in a semicolon.
```sql
sqlite> select 38 as latitude, 122 as longitude, "Berkeley" as name;
38|122|Berkeley
```

A multi-line table can be constructed by `union`, which combines the rows of two tables. The column names of the left table are used in the constructed table. Spacing within a line does not affect the result.
```sql
sqlite> select 38 as latitude, 122 as longitude, "Berkeley" as name union
   ...> select 42,             71,               "Cambridge"        union
   ...> select 45,             93,               "Minneapolis";
38|122|Berkeley
42|71|Cambridge
45|93|Minneapolis
```

A table can be given a name using a `create table` statement. While this statement can also be used to create empty tables, we will focus on the form that gives a name to an existing table defined by a select statement.
```sql
create table cities as
<table>
```
The result of a `select` statement is displayed to the user, but not stored. `create`statements do store tables.

Once a table is named, that name can be used in a from clause within a select statement. All columns of a table can be displayed using the special select * form: `select * from cities`.

### `select` Statements

A `select` statement defines a new table either by listing the values in a single row or, more commonly, by projecting an existing table using a from clause:
```sql
select [column description] from [existing table name]
```
The columns of the resulting table are described by a comma-separated list of expressions that are each evaluated for each row of the existing input table:
```sql
sqlite> select name, 60*abs(latitude-38) from cities;
Berkeley|0
Cambridge|240
Minneapolis|420
```

**Column descriptions** are expressions in a language that shares many properties with Python. Expressions can contain function calls and arithmetic operators. Expressions can
- Combine values: `+, -, *, /, %, and, or` (`/` is integer division).
- Transform values: `abs, round, not, -`.
- Compare values: `<, <=, >, >=, <>, != =` (two different forms of expressing inequlity, `<>` and `!=`).
- Describe evaluation order: `(, )`.
- Create strings: strings are characters enclosed by `" "`. To add strings, `str1 || str2`. To select substrings, `substr(my_str, starting_position, length_substr)`. To select the position a single character, `instr(my_str, character)`.
Names in these expressions, such as `latitude` above, evaluate to the column value in the row being projected.

Optionally, each expression can be followed by the keyword `as` and a column name. When the entire table is given a name, it is often helpful to give each column a name so that it can be referenced in future `select` statements. Columns described by a simple name are named automatically.

**Where Clauses**. A select statement can also include a `where` clause with a filtering expression. This expression filters the rows that are projected. Only a row for which the filtering expression evaluates to a true value will be used to produce a row in the resulting table.

**Order Clauses**. A `select` statement can also express an ordering over the resulting table. An `order` clause contains an ordering expression that is evaluated for each unfiltered row. The resulting values of this expression are used as a sorting criterion for the result table.

### Joins

Databases typically contain multiple tables, and queries can require information contained within different tables to compute a desired result. Data are combined by *joining* multiple tables together into one, a fundamental operation in database systems.

When tables are joined, the resulting table contains a new row for each combination of rows in the input tables. If two tables are joined and the left table has $m$ rows and the right table has $n$ rows, then the joined table will have $m \cdot n$ rows and $m + n$ columns. Joins are expressed in SQL by separating table names by commas in the from clause of a `select` statement.

Joins are typically accompanied by a `where` clause that expresses a relationship between the two tables. The `where` clause can select for rows in the joined table in which these values are equal. In SQL, numeric equality is tested with a single `=` symbol. 

So far our syntax is:
```sql
select [columns] from [table] where [condition] order by [order];
```
with `[columns]` is of the form `[expression] as [name], [expression] as [name]...`.

Tables may have overlapping column names, and so we need a method for disambiguating column names by table. A table may also be joined with itself, and so we need a method for disambiguating tables. To do so, SQL allows us to give **aliases** to tables within a `from` clause using the keyword `as` and to refer to a column within a particular table using a dot expression.
```sql
sqlite> select a.city, b.city, a.temp - b.temp
   ...>        from temps as a, temps as b where a.city < b.city;
Berkeley|Chicago|10
Berkeley|Minneapolis|15
Chicago|Minneapolis|5
```

### Aggregation and Grouping

A `select` statement can perform aggregation operations over multiple rows. The **aggregate functions** `max`, `min`, `count`, `avg` and `sum` return the maximum, minimum, number, average, and sum of the values in a column clause. Multiple aggregate functions can be applied to the same set of rows by defining more than one column. Only columns that are included by the where clause are considered in the aggreagation. The `distinct` keyword ensures that no repeated values in a column are included in the aggregation. Each of these select statements has produced a table with a single row.

We have to be careful in selecting single values in conjunction with aggregations. Sometimes it can return meaningful values, but in other cases do not (in particular, this is the case in which several columns satisfy the aggregation).

The `group by` and `having` clauses of a `select` statement are used to partition rows into groups and select only a subset of the groups. The number of groups is the number of unique values of the expression that follows `group by`. Any aggregate functions in the `having` clause or column description will apply to each group independently, rather than the entire set of rows in the table.

```sql
sqlite> create table animals as
  ....>   select "dog" as name, 4 as legs, 20 as weight union
  ....>   select "cat"        , 4        , 10           union
  ....>   select "ferret"     , 4        , 10           union
  ....>   select "t-rex"      , 2        , 12000        union
  ....>   select "penguin"    , 2        , 10           union
  ....>   select "bird"       , 2        , 6;
sqlite> select legs, max(weight) from animals group by legs;
2|12000
4|20
sqlite> select weight from animals group by weight having count(*)>1;
10
```

The syntax for grouping and aggregation of groups is
```sql
select [columns] from [table] group by [expression] having [expression]
```

Multiple columns and full expressions can appear in the `group by` clause, and groups will be formed for every unique combination of values that result. Typically, the expression used for grouping also appears in the column description, so that it is easy to identify which result row resulted from each group.
```sql
sqlite> select max(name), legs, weight from animals group by legs, weight
  ....>   having max(weight) < 100;
bird|2|6
penguin|2|10
ferret|4|10
dog|4|20
```

A `having` clause can contain the same filtering as a `where` clause, but can also include calls to aggregate functions. For the fastest execution and clearest use of the language, a condition that filters individual rows based on their contents should appear in a `where` clause, while a `having` clause should be used only when aggregation is required in the condition.

### `create table` and `drop table`

The `create table` statement creates a new table in our database. As we saw earlier, we can combine the `create table` statement with the `select` statement to give a name to an existing table. We can also use the `create table` statement along with a list of column names to create an empty table. For each column, we can optionally include the `unique` keyword, which indicates that the column can only contain unique values, or the `default` keyword, which gives a default value for an item in the column. For the entire `create table` statement, including the optional `if not exists` clause will prevent an error if we attempt to create duplicate tables.

The `drop table` statement deletes a table from our database. Including the optional `if exists` clause will prevent an error if we attempt to drop a non-existing table.

### Modifying Tables

The `insert into` statement allows us to add rows to a table in our database. In particular, we can insert values into all columns of our table, or we can add to one specific column, which will set the other columns to their default values. By combining the `insert into` and `select` statements, we can add the rows of an existing table to our table.
```sql
sqlite> create table primes (n unique, prime default 1);
sqlite> insert into primes values (2, 1), (3, 1);
sqlite> select * from primes;
2|1
3|1
sqlite> insert into primes(n) values (4), (5);
sqlite> select * from primes;
2|1
3|1
4|1
5|1
```

The `update` statement sets all entries in certain columns of a table to new values for a subset of rows as indicated by an optional `where` clause. We can update all rows by omitting the optional `where` clause.

The `delete from` statement deletes a subset of rows of a table as indicated by an optional `where` clause. If we do not include a `where` clause, then we will delete all rows, but an empty table would remain in our database.
```sql
sqlite> update primes set prime = 0 where n > 2 and n % 2 = 0;
sqlite> update primes set prime = 0 where n > 3 and n % 3 = 0;
sqlite> select * from primes;
2|1
3|1
4|0
5|1
6|0
7|1
8|0
9|0
sqlite> delete from primes where prime = 0;
sqlite> select * from primes;
2|1
3|1
5|1
7|1
```

See slides for a flowchart diagram of the syntax of the previous statements.

### Python and SQL

Watch https://www.youtube.com/watch?v=VJYBSf-whT4&list=PL6BsET-8jgYW0QqB41ay0bjhNeY1kMdYR&index=4, https://www.youtube.com/watch?v=tjlgWZ82kLA&list=PL6BsET-8jgYW0QqB41ay0bjhNeY1kMdYR&index=5 and https://www.youtube.com/watch?v=2Ak8LgTS4R4&list=PL6BsET-8jgYW0QqB41ay0bjhNeY1kMdYR&index=6.